In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
TrainData = pd.read_csv("5001train.csv")
TestData  = pd.read_csv("5001test.csv")
from pandas.core.frame import DataFrame
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# supposed maximal n_jobs is 16
for i in range(400):
    if TrainData['n_jobs'][i] == -1:
        TrainData['n_jobs'][i] = 16

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
#print real time
result=[]
for i in  range(0, 400):
    X1,Y1=make_classification(n_samples = TrainData.iat[i,7], n_features= TrainData.iat[i,8] , n_classes = TrainData.iat[i,9]  , n_clusters_per_class = TrainData.iat[i,10] ,n_informative = TrainData.iat[i,11], flip_y= TrainData.iat[i,12] , scale= TrainData.iat[i,13])
    import time
    from sklearn.linear_model import SGDClassifier
    clf = SGDClassifier(penalty = TrainData.iat[i,1], l1_ratio = TrainData.iat[i,2], alpha = TrainData.iat[i,3], max_iter = TrainData.iat[i,4], random_state = TrainData.iat[i,5], n_jobs = TrainData.iat[i,6])
    start = time.time()  
    clf.fit(X1, Y1)
    elapsed = (time.time() - start)
    result.append(elapsed)  
print(i,result)

399 [0.5287737846374512, 3.108427047729492, 0.3647949695587158, 1.022508144378662, 0.716839075088501, 12.551417112350464, 2.380812168121338, 5.002321004867554, 0.8260650634765625, 3.179537296295166, 2.6557888984680176, 1.1288769245147705, 0.8204910755157471, 3.0569889545440674, 1.7499809265136719, 0.10687422752380371, 1.5837430953979492, 0.41169190406799316, 1.44844388961792, 4.019684076309204, 0.10852193832397461, 0.4156479835510254, 0.6117250919342041, 0.6210618019104004, 2.3756959438323975, 0.5251290798187256, 2.159442186355591, 0.4118032455444336, 1.2265751361846924, 2.869878053665161, 1.6649539470672607, 0.5180611610412598, 0.20778799057006836, 5.014949798583984, 0.7200651168823242, 1.962796926498413, 5.246016979217529, 0.20919394493103027, 19.081645965576172, 0.10424518585205078, 11.385681867599487, 3.3038699626922607, 0.718425989151001, 0.4190359115600586, 0.6203067302703857, 2.5779459476470947, 5.815141916275024, 15.309467792510986, 19.17103624343872, 0.10719680786132812, 5.005

In [4]:
pd.set_option('display.max_rows', None)
TrainData['time_pre']=result

In [5]:
TrainData1 = pd.get_dummies(TrainData, columns = ['penalty'])

In [6]:
pd.set_option('display.max_rows', None)
f=pd.DataFrame({"id":TrainData['id'], "time":TrainData['time'],"time_pred":TrainData1['time_pre']})

In [7]:
TrainData2 = TrainData1[['l1_ratio' , 'alpha' , 'max_iter' , 'n_jobs' , 'n_samples' , 'n_features' , 'n_classes' , 'n_clusters_per_class'  , 'flip_y' , 'scale', 'time_pre', 'penalty_elasticnet', 'penalty_l1','penalty_l2','penalty_none']]

In [8]:
n_samplefeature = TrainData2.n_samples*TrainData2.n_features
TrainData2['nsamplefeature'] =n_samplefeature
TrainData2.drop('n_samples',axis=1, inplace=True)
TrainData2.drop('n_features',axis=1, inplace=True)
n_classes1 = TrainData2.n_classes*TrainData2.n_clusters_per_class
TrainData2['n_classes1'] =n_classes1
TrainData2.drop('n_classes',axis=1, inplace=True)
TrainData2.drop('n_clusters_per_class',axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
TrainData2=TrainData2.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x))) 

In [17]:
model = Sequential()
model.add(Dense(output_dim=1000, input_dim=13, init='uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(output_dim=1, input_dim=1000,init='uniform'))
model.compile(loss='mean_squared_error', optimizer='adam')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=13, activation="relu", units=1000, kernel_initializer="uniform")`
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=1000, units=1, kernel_initializer="uniform")`
  after removing the cwd from sys.path.


In [11]:
for i in range(100):
    if TestData['n_jobs'][i] == -1:
        TestData['n_jobs'][i] = 16

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
result2=[]
for i in  range(0, 100):
    X2,Y2=make_classification(n_samples = TestData.iat[i,7], n_features= TestData.iat[i,8] , n_classes = TestData.iat[i,9]  , n_clusters_per_class = TestData.iat[i,10] ,n_informative = TestData.iat[i,11], flip_y= TestData.iat[i,12] , scale= TestData.iat[i,13])
    import time
    from sklearn.linear_model import SGDClassifier
    clf = SGDClassifier(penalty = TestData.iat[i,1], l1_ratio = TestData.iat[i,2], alpha = TestData.iat[i,3], max_iter = TestData.iat[i,4], random_state = TestData.iat[i,5], n_jobs = TestData.iat[i,6])
    start = time.time()  
    clf.fit(X2, Y2)
    elapsed = (time.time() - start)
    result2.append(elapsed)  
print(i,result2)

99 [1.8439350128173828, 15.163701057434082, 0.9275729656219482, 2.476964235305786, 3.417428970336914, 13.073661804199219, 2.782838821411133, 1.0379631519317627, 12.739984035491943, 0.49919795989990234, 5.460515975952148, 42.91088795661926, 1.1511502265930176, 23.30795693397522, 0.15194082260131836, 0.7323782444000244, 1.0313589572906494, 9.477136135101318, 2.778496026992798, 2.071234941482544, 0.3114309310913086, 0.3738260269165039, 0.7225978374481201, 0.822598934173584, 2.047658920288086, 3.295024871826172, 1.657109022140503, 9.006521224975586, 1.4531900882720947, 2.6983988285064697, 18.56486225128174, 2.9110682010650635, 1.348106861114502, 6.547025918960571, 16.632874011993408, 1.1218318939208984, 5.332128047943115, 4.603480100631714, 1.3356549739837646, 2.2984557151794434, 9.739620685577393, 8.502506017684937, 1.4415867328643799, 14.61195993423462, 1.2290401458740234, 1.5208640098571777, 5.639392137527466, 1.3312718868255615, 9.06163477897644, 1.9462080001831055, 0.321505069732666, 

In [12]:
pd.set_option('display.max_rows', None)
TestData['time_pre']=result2
TestData1 = pd.get_dummies(TestData, columns = ['penalty'])

In [13]:
TestData2 = TestData1[['l1_ratio' , 'alpha' , 'max_iter'  , 'n_jobs' , 'n_samples' , 'n_features' , 'n_classes' , 'n_clusters_per_class' , 'flip_y' , 'scale', 'time_pre', 'penalty_elasticnet', 'penalty_l1','penalty_l2','penalty_none']]

In [14]:
n_samplefeature = TestData2.n_samples*TestData2.n_features
TestData2['nsamplefeature'] =n_samplefeature
TestData2.drop('n_samples',axis=1, inplace=True)
TestData2.drop('n_features',axis=1, inplace=True)
TestData2['n_classes1'] =n_classes1
TestData2.drop('n_classes',axis=1, inplace=True)
TestData2.drop('n_clusters_per_class',axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [15]:
TestData2=TestData2.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [18]:
model.fit(TrainData2, TrainData1.time, nb_epoch= 170, batch_size=10,  verbose=2)
# evaluate the model

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/170
 - 1s - loss: 30.6006
Epoch 2/170
 - 0s - loss: 21.8562
Epoch 3/170
 - 0s - loss: 17.6311
Epoch 4/170
 - 0s - loss: 14.6310
Epoch 5/170
 - 0s - loss: 12.1326
Epoch 6/170
 - 0s - loss: 10.1143
Epoch 7/170
 - 0s - loss: 8.7777
Epoch 8/170
 - 0s - loss: 7.4324
Epoch 9/170
 - 0s - loss: 6.1283
Epoch 10/170
 - 0s - loss: 5.4824
Epoch 11/170
 - 0s - loss: 4.7884
Epoch 12/170
 - 0s - loss: 4.3566
Epoch 13/170
 - 0s - loss: 3.6856
Epoch 14/170
 - 0s - loss: 3.4334
Epoch 15/170
 - 0s - loss: 2.9705
Epoch 16/170
 - 0s - loss: 2.9012
Epoch 17/170
 - 0s - loss: 2.4204
Epoch 18/170
 - 0s - loss: 2.4432
Epoch 19/170
 - 0s - loss: 1.9434
Epoch 20/170
 - 0s - loss: 1.8175
Epoch 21/170
 - 0s - loss: 1.8308
Epoch 22/170
 - 0s - loss: 1.5964
Epoch 23/170
 - 0s - loss: 1.6265
Epoch 24/170
 - 0s - loss: 1.4979
Epoch 25/170
 - 0s - loss: 1.3656
Epoch 26/170
 - 0s - loss: 1.2759
Epoch 27/170
 - 0s - loss: 1.2626
Epoch 28/170
 - 0s - loss: 1.2285
Epoch 29/170
 - 0s - loss: 1.1659
Epoch 30/170
 - 0

In [19]:
y1=model.predict(TestData2)
y1

array([[ 1.01831365e+00],
       [ 8.93522930e+00],
       [ 2.52116859e-01],
       [ 7.50527442e-01],
       [ 1.44519663e+00],
       [ 8.07172871e+00],
       [ 1.43982124e+00],
       [ 3.30507785e-01],
       [ 1.26811934e+01],
       [ 3.15798879e-01],
       [ 3.80253482e+00],
       [ 1.20189781e+01],
       [ 5.31290889e-01],
       [ 2.44364128e+01],
       [ 1.22383222e-01],
       [ 1.66564018e-01],
       [ 1.19008034e-01],
       [ 4.44333410e+00],
       [ 1.59211445e+00],
       [ 8.88595641e-01],
       [ 1.52544022e-01],
       [ 3.03361595e-01],
       [ 2.77114153e-01],
       [-1.13095090e-01],
       [ 6.22438610e-01],
       [ 1.10185862e+00],
       [ 1.67654648e-01],
       [ 2.16837668e+00],
       [ 1.07822692e+00],
       [ 2.64055705e+00],
       [ 2.08564644e+01],
       [ 1.06620717e+00],
       [ 2.91284442e-01],
       [ 4.60457325e+00],
       [ 5.65910578e+00],
       [ 4.97331589e-01],
       [ 3.79556727e+00],
       [ 2.03920722e+00],
       [ 4.6

In [20]:
y2=[]
for i in range(0,100):
    y2.append(y1[i][0])
y2

[1.0183136,
 8.935229,
 0.25211686,
 0.75052744,
 1.4451966,
 8.071729,
 1.4398212,
 0.33050779,
 12.681193,
 0.31579888,
 3.8025348,
 12.018978,
 0.5312909,
 24.436413,
 0.12238322,
 0.16656402,
 0.119008034,
 4.443334,
 1.5921144,
 0.88859564,
 0.15254402,
 0.3033616,
 0.27711415,
 -0.11309509,
 0.6224386,
 1.1018586,
 0.16765465,
 2.1683767,
 1.0782269,
 2.640557,
 20.856464,
 1.0662072,
 0.29128444,
 4.6045732,
 5.659106,
 0.4973316,
 3.7955673,
 2.0392072,
 0.46364757,
 1.253936,
 3.5880315,
 6.4411044,
 0.32796627,
 2.641438,
 0.600651,
 2.8551648,
 3.217547,
 0.27197307,
 4.9211087,
 0.51317906,
 -0.0393309,
 24.605663,
 -0.068979785,
 6.112612,
 9.054009,
 0.2052533,
 0.026678175,
 0.76005584,
 0.6151643,
 1.1721332,
 1.0131934,
 3.9937482,
 0.343066,
 6.4040976,
 2.9858065,
 7.7367144,
 0.45373583,
 0.07999453,
 10.431518,
 0.6286854,
 0.29061413,
 0.46741068,
 6.8835154,
 0.1505625,
 5.4911757,
 6.3800583,
 0.81776947,
 2.0305254,
 3.049072,
 0.7854623,
 5.719866,
 3.179054,


In [21]:
for i in range(100):
    if y2[i]<0:
        y2[i] = TestData2['time_pre'][i]

In [22]:
pd.DataFrame({"id":TestData['id'],"time":list(y2)}).to_csv('/Users/wyx/Desktop/5001prediction.csv', columns=['id','time'],index=None)